In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Simple valuation with default recipes

This notebook shows how to value a portfolio using default recipes, for an out of the box look at positions and valuations

Attributes
----------
valuation
transactions
recipes
manifests
"""


toggle_code("Hide docstring")

# Valuation

>⚠️ WARNING ⚠️
>
>In order to run this notebook, code in sections 1.1 needs to be uncommented.
>Be aware that this will have system-wide implications for booking transactions (beyond the scope of this notebook) as enabling the "SetTradeToPortfolioRate" field enables it across the entire LUSID domain.

This notebook illustrates an example of how to use [*GetValuation*](https://www.lusid.com/docs/api/#operation/GetValuation) for a simplified call to the valuation engine. This notebook also demonstrates how LUSID can return the cost of trades in the base portfolio currency when trades are booked with different trade currencies to the base portfolio currency. This is possible by accessing FX rates that we load into the quotes store.

## Table of contents
- 1. [Load data](#1.-Load-Data)
   * [1.1 Config Store](#1.1-Config-Store)
   * [1.2 Define Valuation Recipe](#1.2-Define-Valuation-Recipe)
   * [1.3 FX rate](#1.3-FX-rate)
   * [1.4 Instruments](#1.4-Instruments)
   * [1.5 Portfolio](#1.5-Portfolio)
   * [1.6 Transactions](#1.6-Transactions)
   * [1.7 Quotes](#1.7-Quotes)
- 2. [Run valuation](#2.-Run-valuation)
    * [2.1 Single-day](#2.1-Single-day)
    * [2.2 Multi-day subtotals](#2.2-Multi-day-subtotals)
    * [2.3 Multi-day ranges](#2.3-Multi-day-ranges)
- 3. [A2B Report](#3.-A2B-Report)
    * [3.1 A2B Currency Gains](#3.1-A2B-Currency-Gains)

In [2]:
# Import system packages

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python

import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
import lusid_configuration
from lusid_configuration import ConfigurationSetsApi
from lusid_configuration import models as config_models
from fbnsdkutilities import ApiClientFactory as ConfigApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

import os
import pandas as pd
import pytz
from datetime import datetime, timedelta

# Set pandas dataframe display formatting
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")


# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

api_client = api_factory.api_client

lusid_api_url = api_client.configuration.host
configuration_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "configuration"

config_api_factory = ConfigApiClientFactory(
    lusid_configuration,
    token=api_client.configuration.access_token,
    api_url=configuration_api_url,
    app_name="LusidJupyterNotebook",
)

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.6.10884.0


In [3]:
# Define the apis
quotes_api = api_factory.build(lusid.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
instruments_api = api_factory.build(lusid.InstrumentsApi)
transaction_portfolios_api = api_factory.build(lusid.TransactionPortfoliosApi)
sets_api = config_api_factory.build(lusid_configuration.api.ConfigurationSetsApi)
aggregation_api = api_factory.build(lusid.AggregationApi)

## 1. Load Data

In [4]:
# Define Variables
scope = "simple-valuation"
portfolio_code = "global-equities"
recipe_scope="IBOR"
recipe_code="TTPR_NB_Recipe"
start_date = datetime(year=2020, month=8, day=24, tzinfo=pytz.UTC)
end_date = datetime(year=2020, month=8, day=28, tzinfo=pytz.UTC)

transaction_date = datetime(year=2020, month=8, day=3, tzinfo=pytz.UTC)

### 1.1 Config Store

Set the "SetTradeToPortfolioRate" item to True to enable the calculation of the trade to portfolio rate in valuations.
<span style="color:red">(This will need to be uncommented)</span>

(Details on TradeToPortfolioRate property can be found here - https://support.lusid.com/knowledgebase/article/KA-01744/en-us#transaction)

Also, set the scope and code of the "TradeToPortfolioRateRecipe" item (RecipeScope/RecipeCode)

In [5]:
###########
# WARNING #
###########

# Turning on this property is a system-wide change and could affect other processes.

# Uncomment this to turn on TTPR

#Configure "SetTradeToPortfolioRate"

# config_ttpr_response = sets_api.update_configuration_item(
#      type="Shared",
#      scope="system",
#      code="TransactionBooking",
#      key="SetTradeToPortfolioRate",
#      update_configuration_item = config_models.UpdateConfigurationItem(value="True")
#  )

# Configure "TradeToPortfolioRateRecipe" config item
config_ttpr_recipe_response = sets_api.update_configuration_item(
    type="Shared",
    scope="system",
    code="TransactionBooking",
    key="TradeToPortfolioRateRecipe",
    update_configuration_item = config_models.UpdateConfigurationItem(value="IBOR/TTPR_NB_Recipe")
)

### 1.2 Define Valuation Recipe
Define the valuation recipe to be linked to the trade to portfolio rate configuration in the configuration store.

In [6]:
# Create a recipe to perform a valuation
configuration_recipe = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    market=models.MarketContext(
        market_rules=[
            # define how to resolve the quotes
            models.MarketDataKeyRule(
                key='Fx.CurrencyPair.*',
                data_scope=scope,
                supplier='Lusid',
                quote_type='Rate',
                quote_interval='1D.0D',
                field="mid"
            ),
            models.MarketDataKeyRule(
                key='Quote.LusidInstrumentId.*',
                data_scope=scope,
                supplier='Lusid',
                quote_type='Price',
                quote_interval='1D.0D',
                field="mid"

            )
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope='Lusid',
            # This enables FX rate inference
            attempt_to_infer_missing_fx=True,

        ),
    ),
    pricing=models.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": True},
    ),
)

# Upsert recipe to LUSID
upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=models.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

### 1.3 FX rate
Load 6 different FX rate quotes for the different date's booked into this portfolio (see below).

In [7]:
# Define function to create FX rate upsert quote requests for provided currencies and effective date
def spot_request(from_ccy, to_ccy, fx_rate, valuation_date):
            return models.UpsertQuoteRequest(
                       quote_id=models.QuoteId(
                           models.QuoteSeriesId(
                               provider='Lusid',
                               instrument_id=f'{from_ccy}/{to_ccy}',
                               instrument_id_type='CurrencyPair',
                               quote_type='Rate',
                               field='mid'
                           ),
                           effective_at=valuation_date
                       ),
                       metric_value=models.MetricValue(
                           value=fx_rate,
                           unit=f'{from_ccy}/{to_ccy}'
                       ),
                       lineage='None'
            )

In [8]:
# Upsert 6 FX rate quotes
fx_rates = [
    [0.785, transaction_date.isoformat()],
    [0.77, start_date.isoformat()],
    [0.78, (start_date + timedelta(days=1)).isoformat()], 
    [0.79, (start_date + timedelta(days=2)).isoformat()], 
    [0.80, (start_date + timedelta(days=3)).isoformat()], 
    [0.81, (start_date + timedelta(days=4)).isoformat()]
]

request_body = {i:spot_request("USD", "GBP", fx_rates[i][0], fx_rates[i][1]) for i in range(len(fx_rates))}

response = quotes_api.upsert_quotes(scope=scope, request_body=request_body)

### 1.4 Instruments

Read the data from the quotes files, reading the instrument names and identifiers and adding the unique names to LUSID.

In [9]:
instruments_df = pd.read_excel("data/simple-valuation/ftse-100-prices31-Jul-2020-31-Aug-2020.xlsx")[["name", "figi"]].drop_duplicates()
instruments_df.head()

,name,figi
0,SCOTTISH MORTGAGE INV TR PLC,BBG000BFZMY9
21,FRESNILLO PLC,BBG000VH0TC0
42,AVAST PLC,BBG00KW3SK62
63,POLYMETAL INTERNATIONAL PLC,BBG0025RP8F9
84,AVEVA GROUP PLC,BBG000C21Y87


Create a mapping schema for the instruments using the provided FIGIs as the instrument identifiers. The instruments file is then loaded into LUSID.

In [10]:
instrument_mapping = {
    "identifier_mapping": {
        "Figi": "figi"
    },
    "required": {
        "name": "name"
    },
}

In [11]:
# Instruments can be loaded using a dataframe with file_type set to "instruments"
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instruments_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    instrument_scope=scope,
    identifier_mapping=instrument_mapping["identifier_mapping"]
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,101,0,0


In [12]:
us_instruments_df = pd.read_excel("data/simple-valuation/usportfolio.xlsx")[["name", "ticker"]].drop_duplicates()
us_instruments_df.head()

,name,ticker
0,Uber Technologies Inc,UBER
1,Zoom Video Communications Inc,ZM
2,Spotify Technology SA,SPOT
3,Pinterest Inc,PINS
4,Nike Inc,NKE


Create a mapping schema for the instruments using the provided Ticker as the instrument identifiers. The instruments file is then loaded into LUSID.

In [13]:
us_instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "ticker"
    },
    "required": {
        "name": "name"
    },
}

In [14]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=us_instruments_df,
    mapping_required=us_instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    instrument_scope=scope,
    identifier_mapping=us_instrument_mapping["identifier_mapping"]
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,15,0,0


The instruments should now be viewable in the [LUSID webtool](https://www.lusid.com/app/home) (*Data Management* >>> *Instruments*)

### 1.5 Portfolio

Create a portfolio in LUSID by setting up a mapping schema which can then be used to load the relevant contents.

In [15]:
uk_portfolio_df = pd.read_excel("data/simple-valuation/ukportfolio.xlsx")
us_portfolio_df = pd.read_excel("data/simple-valuation/usportfolio.xlsx")
portfolio_df = pd.concat([uk_portfolio_df, us_portfolio_df])
portfolio_df

,portfolio_code,portfolio_name,portfolio_base_currency,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,cash_transactions,Strategy,HoldingType,Commission,Fx_rate,Sector,ticker
0,global-equities,Equity Portfolio,GBP,BBG000BWF7M0,ANGLO AMERICAN PLC,T1,Buy,2020-08-03,2020-08-05,105000,"1,910.60","2,016,700.00",GBP,NaN,Tracker,Equity,"5,041.75",1.00,NaN,NaN
1,global-equities,Equity Portfolio,GBP,BBG000BDCLS8,CRODA INTERNATIONAL PLC,T2,Buy,2020-08-03,2020-08-05,3500,"5,820.00","198,934.00",GBP,NaN,Tracker,Equity,497.33,1.00,NaN,NaN
2,global-equities,Equity Portfolio,GBP,BBG000BD3SC0,ASSOCIATED BRITISH FOODS PLC,T3,Buy,2020-08-03,2020-08-05,51000,"1,786.00","903,500.00",GBP,NaN,Tracker,Equity,"2,258.75",1.00,NaN,NaN
3,global-equities,Equity Portfolio,GBP,BBG000BD6DG6,BARRATT DEVELOPMENTS PLC,T4,Buy,2020-08-03,2020-08-05,57000,524.40,"303,360.00",GBP,NaN,Tracker,Equity,758.40,1.00,NaN,NaN
4,global-equities,Equity Portfolio,GBP,BBG000NSXQ99,BURBERRY GROUP PLC,T5,Buy,2020-08-03,2020-08-05,93000,"1,280.00","1,190,400.00",GBP,NaN,Tracker,Equity,"2,976.00",1.00,NaN,NaN
5,global-equities,Equity Portfolio,GBP,BBG000C0M8X7,UNILEVER PLC,T6,Buy,2020-08-03,2020-08-05,30000,"4,664.00","1,397,770.00",GBP,NaN,Tracker,Equity,"3,494.43",1.00,NaN,NaN
6,global-equities,Equity Portfolio,GBP,BBG000BDY8C0,PEARSON PLC,T7,Buy,2020-08-03,2020-08-05,148000,541.80,"804,170.00",GBP,NaN,Tracker,Equity,"2,010.42",1.00,NaN,NaN
7,global-equities,Equity Portfolio,GBP,BBG000BF46Y8,TESCO PLC,T8,Buy,2020-08-03,2020-08-05,133000,219.20,"292,675.00",GBP,NaN,Tracker,Equity,731.69,1.00,NaN,NaN
8,global-equities,Equity Portfolio,GBP,BBG000BRVH05,WHITBREAD PLC,T9,Buy,2020-08-03,2020-08-05,36000,"2,220.00","799,414.00",GBP,NaN,Tracker,Equity,"1,998.54",1.00,NaN,NaN
9,global-equities,Equity Portfolio,GBP,BBG000BDQGR5,LLOYDS BANKING GROUP PLC,T10,Buy,2020-08-03,2020-08-05,5750000,27.87,"1,599,120.00",GBP,NaN,Tracker,Equity,"3,997.80",1.00,NaN,NaN


In [16]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$GBP",
    },
    "optional": {
        "created": "$2020-01-01T00:00:00+00:00",
        "instrument_scopes": f"${scope}"
                 },
}

In [17]:
# A portfolio can be loaded using a dataframe with file_type = "portfolios"
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=portfolio_df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=[],
    properties_scope="default"
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


### 1.6 Transactions

Create transaction mapping schemas that use the provided FIGI or Ticker identifiers to load the data into LUSID.

In [18]:
uk_transaction_mapping = {
    "identifier_mapping": {
        "Figi": "instrument_id"
    },
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "currency",
        "settlement_date": "txn_settle_date",
    },
    "optional": {},
    "properties": [
        {"scope": "default", "source": "Fx_rate", "target": "TradeToPortfolioRate"},
    ],
}

In [19]:
us_transaction_mapping = {
    "identifier_mapping": {
        "ClientInternal": "ticker"
    },
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "currency",
        "settlement_date": "txn_settle_date",
    },
    "optional": {},
    "properties": [
        {"scope": "default", "source": "Fx_rate", "target": "TradeToPortfolioRate"},
    ],
}

In [20]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=uk_portfolio_df,
    mapping_required=uk_transaction_mapping["required"],
    mapping_optional=uk_transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=uk_transaction_mapping["identifier_mapping"],
    property_columns=uk_transaction_mapping["properties"]
)
succ, failed = format_transactions_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


In [21]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=us_portfolio_df,
    mapping_required=us_transaction_mapping["required"],
    mapping_optional=us_transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=us_transaction_mapping["identifier_mapping"],
    property_columns=us_transaction_mapping["properties"]
)
succ, failed = format_transactions_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


### 1.7 Quotes

Load the source quotes data which contains open and close prices for our Holdings between July 31st and August 31st. This is the pricing data that will be used later in valuation.

In [22]:
uk_quotes_df = pd.read_excel("data/simple-valuation/ftse-100-prices31-Jul-2020-31-Aug-2020.xlsx")
us_quotes_df = pd.read_csv("data/simple-valuation/yahoo-us-prices.csv")

In [23]:
uk_quotes_df.head(1)

,date,ticker,name,figi,Sector,open_price,close_price,currency
0,2020-08-28T00:00:00.000Z,SMT.L,SCOTTISH MORTGAGE INV TR PLC,BBG000BFZMY9,Equity Investment Instruments,960.00,961.50,GBP


In [24]:
us_quotes_df.head(1)

,date,ticker,open_price,close_price,name,currency
0,31/07/2020,UBER,30.40,30.26,Uber Technologies Inc,USD


The current quotes sources data use FIGI or Ticker as the core unique identifiers, but we can call the API and create unique LUSID identifiers (LUID). These will later be used in our valuation call by mapping them against the previously set transactions.

In [25]:
def add_luid_id(data_frame, scope, identifier, id_type):
    client_ids = pd.DataFrame(list(data_frame[identifier].unique()), columns=[identifier])

    # Call lusid_instrument_id to the API for creating the LUIDs
    client_ids["LUID"] = client_ids[identifier].apply(
    lambda x: api_factory.build(lusid.api.InstrumentsApi).get_instrument(
        scope=scope,
        identifier_type=id_type,
        identifier=x).lusid_instrument_id)
    client_ids = client_ids.set_index(identifier)
    data_frame['LUID'] = data_frame[identifier].apply(lambda x: client_ids.loc[x]["LUID"])
    return data_frame

In [26]:
uk_df = add_luid_id(uk_quotes_df, scope, "figi", "Figi")
us_df = add_luid_id(us_quotes_df, scope, "ticker", "ClientInternal")

df = pd.concat([uk_df, us_df],ignore_index=True)

# Check the first one to see that LUID was added
df.head(1)

,date,ticker,name,figi,Sector,open_price,close_price,currency,LUID
0,2020-08-28T00:00:00.000Z,SMT.L,SCOTTISH MORTGAGE INV TR PLC,BBG000BFZMY9,Equity Investment Instruments,960.00,961.50,GBP,LUID_00003DEM


Create a mapping schema for the quotes dataframe to read the using the LUIDs.

In [27]:
quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$LusidInstrumentId",
    "quote_id.effective_at": "date",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "LUID",
    "metric_value.unit": "currency",
}

We can use the end of day close prices for mapping the quotes source data (pricing data assumed to be using "mid" quotes).

In [28]:
quotes_mapping["quote_id.quote_series_id.field"] ="$mid"
quotes_mapping["metric_value.value"] = "close_price"

result = load_from_data_frame(
    api_factory = api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=quotes_mapping,
    mapping_optional={},
    file_type="quotes"
)

succ, failed, errors = format_quotes_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]))

,success,failed,errors
0,2421,0,0


## 2. Run valuation

### 2.1 Single-day

Perform a valuation on the portfolio by using the recipe created above.

- Recipe attributes:

| Price source/supplier | Instrument ID | Quote Type  | Pricing Model |   Calculation    |
|:---------------------:|:-------------:|:-----------:|:-------------:|:----------------:|
|         LUSID         |     LUID      | Price (mid) | Simple Static | Quantity x price |


In [29]:
def aggregation_interval_request(effectiveFrom, effectiveAt, group_by, metrics ):
    return models.ValuationRequest( 
        recipe_id = models.ResourceId(
            scope = recipe_scope,
            code = recipe_code
        ),
         metrics = metrics,
        group_by = group_by,
        valuation_schedule = models.ValuationSchedule(effective_from = effectiveFrom, effective_at=effectiveAt),
        portfolio_entity_ids = [models.PortfolioEntityId(
                                                        scope = scope,
                                                        code = portfolio_code,
                                                        portfolio_entity_type="SinglePortfolio"
        )]
    )

In [30]:
aggregation = aggregation_api.get_valuation(
                                            valuation_request=aggregation_interval_request(
                                                effectiveFrom=None,
                                                effectiveAt=start_date.isoformat(),
                                                group_by=["Instrument/default/Name"],
                                                metrics=[
                                                    models.AggregateSpec("Instrument/default/Name", "Value"),
                                                    models.AggregateSpec("Valuation/PV", "Sum"),
                                                    models.AggregateSpec("Holding/default/Units", "Sum")
                                                ])
            )
pd.DataFrame(aggregation.data)

,Instrument/default/Name,Sum(Valuation/PV),Sum(Holding/default/Units)
0,GLAXOSMITHKLINE PLC,"247,955,600.00","163,000.00"
1,PEARSON PLC,"176,386,000.00","301,000.00"
2,KINGFISHER PLC,"98,640,000.00","360,000.00"
3,VODAFONE GROUP PLC,"58,480,000.00","500,000.00"
4,RENTOKIL INITIAL PLC,"44,753,600.00","83,000.00"
5,PRUDENTIAL PLC,"104,082,500.00","85,000.00"
6,JUST EAT TAKEAWAY,"64,313,000.00","7,300.00"
7,TESCO PLC,"30,191,000.00","133,000.00"
8,BURBERRY GROUP PLC,"200,700,500.00","143,000.00"
9,BP PLC,"18,048,000.00","64,000.00"


### 2.2 Multi-day subtotals

Using the same valuation request, we are also able to inspect the evolution of the portfolio holdings and their value for a custom date range. Using a function that groups by <code>"Analytic/default/ValuationDate"</code> the function can call LUSID for an overall PV of the portfolio as a time series.   

In [31]:
aggregation = aggregation_api.get_valuation(
                                            valuation_request=aggregation_interval_request(
                                                
                                                effectiveFrom=start_date.isoformat(),
                                                effectiveAt=end_date.isoformat(),
                                                group_by=["Analytic/default/ValuationDate"],
                                                metrics=[
                                                    models.AggregateSpec("Analytic/default/ValuationDate", "Value"),
                                                    models.AggregateSpec("Valuation/PvInPortfolioCcy", "Sum")
                                                ])
    )
pd.DataFrame(aggregation.data)

,Analytic/default/ValuationDate,Sum(Valuation/PvInPortfolioCcy)
0,2020-08-24T00:00:00.0000000+00:00,"1,915,179,511.24"
1,2020-08-25T00:00:00.0000000+00:00,"1,895,623,263.62"
2,2020-08-26T00:00:00.0000000+00:00,"1,904,370,976.30"
3,2020-08-27T00:00:00.0000000+00:00,"1,891,959,702.80"
4,2020-08-28T00:00:00.0000000+00:00,"1,882,314,481.03"


### 2.3 Multi-day ranges

Given the new function now holds data across the selected period, we can also apply other types of specifications in the aggregation metrics. For example, we can see the min/max range for the valuation of each holding in the selected time period. This can illustrate how a stock's volatility can drift the exposure of the portfolio, which can be notable for longer periods. 

In [32]:
aggregation = aggregation_api.get_valuation(
                                            valuation_request=aggregation_interval_request(
                                                effectiveFrom=start_date.isoformat(),
                                                effectiveAt=end_date.isoformat(),
                                                group_by=["Instrument/default/Name"],
                                                metrics=[
                                                    models.AggregateSpec("Instrument/default/Name", "Value"),
                                                    models.AggregateSpec("Valuation/PvInReportCcy", "Min"),
                                                    models.AggregateSpec("Valuation/PvInReportCcy", "Max"), 
                                                ]
                                            )
)
pd.DataFrame(aggregation.data)                                   

,Instrument/default/Name,Min(Valuation/PvInReportCcy),Max(Valuation/PvInReportCcy)
0,GLAXOSMITHKLINE PLC,"239,936,000.00","247,955,600.00"
1,PEARSON PLC,"168,379,400.00","176,386,000.00"
2,KINGFISHER PLC,"97,308,000.00","101,232,000.00"
3,VODAFONE GROUP PLC,"55,720,000.00","58,480,000.00"
4,RENTOKIL INITIAL PLC,"44,239,000.00","44,952,800.00"
5,PRUDENTIAL PLC,"101,915,000.00","104,082,500.00"
6,JUST EAT TAKEAWAY,"61,422,200.00","64,313,000.00"
7,TESCO PLC,"29,087,100.00","30,191,000.00"
8,BURBERRY GROUP PLC,"200,700,500.00","207,493,000.00"
9,BP PLC,"16,908,800.00","18,048,000.00"


## 3. A2B Report

For breaking down the profit and loss occurred in a given period, LUSID offers an A2B report, see [here](https://www.lusid.com/docs/api#operation/GetA2BData) for documentation.

The AtoB report will give a breakdown of each holding's P&L, including the flows (transactions) that have occurred in the selected time window as well as totals at the start and end of the window. Within the report you will find the following details:

 - **Market Value (Start)**: The market value at the start of the window in the portfolio base currency (GBP).
 - **Gains**: Capital Gain, Gain due to asset appreciation based on available market data and the Currency Gain from the FX rates.
 - **Flows**: Any transaction activity occurring in the period.

Note, the following table has been filtered down to some core fields.

In [33]:
# Request A2B Movements report
a2b = transaction_portfolios_api.get_a2_b_data(
    scope=scope,
    code=portfolio_code,
    from_effective_at=start_date,
    to_effective_at=end_date,
    recipe_id_scope=recipe_scope,
    recipe_id_code=recipe_code,
    property_keys=["Instrument/default/Name"]
)

In [34]:
# Rename A2B columns in df
rename_cols = {
    "portfolio_id.scope": "Portfolio Scope",
    "portfolio_id.code": "Portfolio Code",
    "holding_type": "Holding Type",
    "instrument_uid": "Instrument ID",
    "Name(default-Properties)": "Instrument Name",
    "start.holding_currency.currency": "Holding Currency",
    "start.holding_currency.total": "Market Value Holding Currency (Start)",
    "start.portfolio_currency.total": "Market Value Portfolio Currency (Start)",
    "start.portfolio_currency.components.MarketGain": "P&L - Since inception (Start)",
    "start.portfolio_currency.components.CurrencyGain": "Currency Gain - Since inception (Start)",
    "gains.portfolio_currency.total":"P&L (Window)",
    "gains.portfolio_currency.components.DeltaMarketGain": "Market Gain (Window)",
    "gains.portfolio_currency.components.DeltaCurrencyGain": "Currency Gain (Window) ",
    "end.portfolio_currency.total": "Market Value (End)",
    "end.portfolio_currency.components.MarketGain": "P&L - Since inception (End)",
    "end.portfolio_currency.components.CurrencyGain": "Currency Gain - Since inception (End)"
}

# Format and filter A2B
a2b_df = lusid_response_to_data_frame(a2b, rename_properties=True)
col_filters = rename_cols.keys()
a2b_df = a2b_df[col_filters].copy()
a2b_df.rename(columns=rename_cols, inplace=True)
a2b_df

,Portfolio Scope,Portfolio Code,Holding Type,Instrument ID,Instrument Name,Holding Currency,Market Value Holding Currency (Start),Market Value Portfolio Currency (Start),P&L - Since inception (Start),Currency Gain - Since inception (Start),P&L (Window),Market Gain (Window),Currency Gain (Window),Market Value (End),P&L - Since inception (End),Currency Gain - Since inception (End)
0,simple-valuation,global-equities,B,CCY_GBP,GBP,GBP,"-21,851,056.00","-21,851,056.00",NaN,NaN,NaN,NaN,NaN,"-21,851,056.00",NaN,NaN
1,simple-valuation,global-equities,B,CCY_USD,USD,USD,"-502,500.00","-386,925.00",NaN,"7,537.50","-20,100.00",NaN,"-20,100.00","-407,025.00",NaN,"-12,562.50"
2,simple-valuation,global-equities,P,LUID_00003DAS,Aflac Inc,USD,"24,297.00","18,708.69","8,698.69",-195.00,708.63,188.63,520.00,"19,417.32","8,887.32",325.00
3,simple-valuation,global-equities,P,LUID_00003DAT,Harmony Gold Mining Co,USD,"1,776.00","1,367.52","-10,182.48",-225.00,29.73,-570.27,600.00,"1,397.25","-10,752.75",375.00
4,simple-valuation,global-equities,P,LUID_00003DAU,American Tower Corp,USD,"196,352.00","151,191.04","89,591.04","-1,200.00","9,493.52","6,293.52","3,200.00","160,684.56","95,884.56","2,000.00"
5,simple-valuation,global-equities,P,LUID_00003DAV,Spotify Technology SA,USD,"40,369.50","31,084.52","7,984.52",-450.00,"1,964.70",764.70,"1,200.00","33,049.22","8,749.22",750.00
6,simple-valuation,global-equities,P,LUID_00003DAW,Carnival Corp,USD,"6,456.00","4,971.12","-10,428.88",-300.00,251.76,-548.24,800.00,"5,222.88","-10,977.12",500.00
7,simple-valuation,global-equities,P,LUID_00003DAX,Diageo PLC,USD,"48,370.00","37,244.90","-3,180.10",-787.50,"1,149.50",-950.50,"2,100.00","38,394.40","-4,130.60","1,312.50"
8,simple-valuation,global-equities,P,LUID_00003DAZ,Nike Inc,USD,"27,957.50","21,527.28","-7,347.72",-562.50,917.82,-582.18,"1,500.00","22,445.10","-7,929.90",937.50
9,simple-valuation,global-equities,P,LUID_00003DB0,Apache Corp,USD,"12,291.00","9,464.07","-23,260.93",-637.50,588.03,"-1,111.97","1,700.00","10,052.10","-24,372.90","1,062.50"


### 3.1 A2B Currency Gains

This A2B time window runs between 24th of August 2020 at 01:00 and 28th of August 2020 at 01:00.

In [35]:
#UBER
a2b_df[a2b_df["Instrument Name"] == 'Uber Technologies Inc']

,Portfolio Scope,Portfolio Code,Holding Type,Instrument ID,Instrument Name,Holding Currency,Market Value Holding Currency (Start),Market Value Portfolio Currency (Start),P&L - Since inception (Start),Currency Gain - Since inception (Start),P&L (Window),Market Gain (Window),Currency Gain (Window),Market Value (End),P&L - Since inception (End),Currency Gain - Since inception (End)
16,simple-valuation,global-equities,P,LUID_00003DB7,Uber Technologies Inc,USD,"1,552.00","1,195.04",-729.96,-37.50,133.36,33.36,100.00,"1,328.40",-696.60,62.50



This Uber position comes from the opening transaction of USD 2,500.00 (50 units @ USD 50.00 per)

* **Start**:
  * The Currency Gain - Since inception (Start) comes from the opening transaction price of the position USD 2,500.00 multiplied by the USD/GBP FX rate of 0.77 on the 24th of August 2020 at 01:00. Then we calculate the value in GBP on the date of the opening transactions, 3rd of August 2020 at 01:00 multiplied by the USD/GBP FX rate of 0.785 at the time. Due to this differences in FX rates we get a Currency Gain of GBP -37.50.



|                  |  USD Cost   |  FX rate  |      GBP      |
|:----------------:|:-----------:|:---------:|:-------------:|
| 24th August 2020 | 2,500.00 |   0.77    |  1,925.00  |
| 3rd August 2020  | 2,500.00 |   0.785   |  1,962.50  |

$$
{GBP}\,\, 1,925.00  - 1,962.50 = -37.5
$$



* **Gains**:
  *  We get a Currency Gain of GBP 100.00 in the (Window) due to the difference in FX rate from the 24th of August 2020 and the 28th August 2020.

|                  |  USD Cost   | FX rate |     GBP     |
|:----------------:|:-----------:|:-------:|:-----------:|
| 28th August 2020 | 2,500.00 |  0.81   | 2,025.00 |
| 24th August 2020 | 2,500.00 |  0.77   | 1,925.00 |

$$
{GBP}\,\, 2,025.00 - 1,925.00 = 100.00
$$



* **End**
  * The Currency Gain - Since inception (End) shows the sum of all the activities for this position, showing a Currency Gain of GBP 62.50.
  
$$
{GBP}\,  -37.5 + 100.00 = 62.50
$$


